In [13]:
import copy
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import get_cmap
from formation_map import get_formation
from genetic_algorithm import initialise_individual, select_individuals, mutate_individual, crossover_individuals
from plotting import animate
from speciation import speciate, penalise_overpopulation, get_genetic_diversity


In [2]:
cmap = get_cmap("viridis")

In [3]:
# Matplotlib default changes.
mpl.rcParams["figure.facecolor"] = "black"
mpl.rcParams["figure.figsize"] = (12, 10)
mpl.rcParams["axes.facecolor"] = "black"
mpl.rcParams["axes.edgecolor"] = "white"
mpl.rcParams["axes.labelcolor"] = "white"
mpl.rcParams["font.size"] = 14
mpl.rcParams["grid.color"] = "white"
mpl.rcParams["savefig.facecolor"] = "black"
mpl.rcParams["text.color"] = "white"
mpl.rcParams["xtick.color"] = "white"
mpl.rcParams["ytick.color"] = "white"
mpl.rcParams["hatch.color"] = "white"
mpl.rcParams["lines.color"] = "white"
mpl.rcParams["legend.edgecolor"] = "none"
mpl.rc('animation', html='html5')

In [4]:
np.random.seed(42)

In [17]:
N_SAMPLES = 10000

N_INDIVIDUALS = 100
N_GENERATIONS = 1000

REPLACE_MUTATION_PROBABILITY = 0.75
SWAP_MUTATION_PROBABILITY = 0.2
SWITCH_MUTATION_PROBABILITY = 0.05
CROSSOVER_PROBABILITY = 0.05

RATING_WEIGHT = 1
CHEMISTRY_WEIGHT = 0.75
PRICE_WEIGHT = 1.5

SPECIATION = True
SPECIATION_THRESHOLD = 2.5

In [15]:
formation = get_formation("5221")

In [7]:
sample_individuals = [initialise_individual(formation) for _ in range(N_SAMPLES)]

In [8]:
ratings = [s.fill().get_rating() for s in sample_individuals]
rating_mean = np.mean(ratings)
rating_std = np.std(ratings)

chemistries = [s.fill().get_chemistry() for s in sample_individuals]
chemistry_mean = np.mean(chemistries)
chemistry_std = np.std(chemistries)

prices = [s.fill().get_price() for s in sample_individuals]
price_mean = np.mean(prices)
price_std = np.std(prices)

In [9]:
def loss(formation):
    rating = (formation.get_rating() - rating_mean) / rating_std # normalised rating.
    chemistry = (formation.get_chemistry() - chemistry_mean) / chemistry_std # normalised chemistry.
    price = -(formation.get_price() - price_mean) / price_std # negative, normalised price.
    return RATING_WEIGHT * rating + CHEMISTRY_WEIGHT * chemistry + PRICE_WEIGHT * price

In [ ]:
fitness_log = []
rating_log = []
chemistry_log = []
price_log = []
champion_log = []
diversity_log = []

champion = None
population = [initialise_individual(formation) for _ in range(N_INDIVIDUALS)]
for individual in population:
    individual.evaluate(loss)


for i in range(N_GENERATIONS):
    if SPECIATION:
        species = speciate(population, SPECIATION_THRESHOLD)
        diversity_log.append(get_genetic_diversity(species))
        penalise_overpopulation(species)
    population = select_individuals(population)
    for j in range(int(len(population)/2)):
        individuals = population[2*j:2*j+2]
        if np.random.uniform() < CROSSOVER_PROBABILITY:
            crossover_individuals(*individuals, formation)
        for individual in individuals:
            mutate_individual(individual, REPLACE_MUTATION_PROBABILITY, SWAP_MUTATION_PROBABILITY, SWITCH_MUTATION_PROBABILITY)
            individual.evaluate(loss)
    challenger = population[np.argmax(population)]
    new_champion = False
    if not champion or challenger > champion:
        champion = copy.copy(challenger)
        champion_log.append(champion)
        new_champion = True
    champion.fill()
    fitness_log.append(champion.fitness)
    rating_log.append(champion.formation.get_rating())
    chemistry_log.append(champion.formation.get_chemistry())
    price_log.append(champion.formation.get_price())
    if new_champion:   
        print("TURN {}".format(i))
        print("PLAYERS:")
        for k, v in champion.player_map.items():
            print("\t{}: {}".format(k, v))
        print("FITNESS: {0:.2f}".format(champion.fitness))
        print("RATING: {}".format(champion.formation.get_rating()))
        print("CHEMISTRY: {}".format(champion.formation.get_chemistry()))
        print("PRICE: {}".format(int(champion.formation.get_price())))
        print("")
        
print("FINAL CHAMPION")
print("PLAYERS:")
for k, v in champion.player_map.items():
    print("\t{}: {}".format(k, v))
print("FITNESS: {0:.2f}".format(champion.fitness))
print("RATING: {}".format(champion.formation.get_rating()))
print("CHEMISTRY: {}".format(champion.formation.get_chemistry()))
print("PRICE: {}".format(int(champion.formation.get_price())))
print("")

TURN 0
PLAYERS:
	GK: Player(id=12873, name=Gamonal, overall=68, position=GK, price=200, nationality=Chile, league=Campeonato Scotiabank, club=Club Deportes Temuco)
	LWB: Player(id=8379, name=Meredith, overall=71, position=LB, price=750, nationality=Australia, league=EFL Championship, club=Millwall)
	CB1: Player(id=9382, name=Calisir, overall=67, position=CB, price=300, nationality=Sweden, league=Allsvenskan, club=IFK Göteborg)
	CB2: Player(id=10970, name=Pehrsson, overall=64, position=CB, price=500, nationality=Sweden, league=Allsvenskan, club=IK Sirius)
	CB3: Player(id=1178, name=Rúben Dias, overall=78, position=CB, price=950, nationality=Portugal, league=Liga NOS, club=SL Benfica)
	RWB: Player(id=1255, name=Kaderábek, overall=79, position=RWB, price=950, nationality=Czech Republic, league=Bundesliga, club=TSG 1899 Hoffenheim)
	CM1: Player(id=13907, name=Račić, overall=75, position=CDM, price=500, nationality=Serbia, league=LaLiga Santander, club=Valencia CF)
	CM2: Player(id=1248, nam

In [ ]:
with open("../input/champion_log.p", "wb") as f:
    pickle.dump(champion_log, f)

In [ ]:
x = np.arange(0, N_GENERATIONS)
fig, axs = plt.subplots(4, 1, figsize=(16, 12))

for ax, data, title in zip(axs, [fitness_log, rating_log, chemistry_log, price_log], ["Fitness", "Rating", "Chemistry", "Price"]):
    ax.plot(x, data, linewidth=3, color=cmap(1.))
    ax.set_title(label=title)
    ax.set_xlabel("Generation")    
    
fig.tight_layout()

plt.savefig("../images/fitness_curve.png", dpi=120)